In [1]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [4]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import numpy as np
import h5py
import re

from nltk.translate.bleu_score import corpus_bleu

import pickle

def create_caption_greedy(model, tokenizer, feature, max_len):
  start = 'sq'
  for i in range(max_len):
    seq = tokenizer.texts_to_sequences([start])[0]
    seq = pad_sequences([seq],maxlen = max_len)

    pred = model.predict([feature, seq],verbose = 0)
    
    pred = np.argmax(pred)
    pred_word = 'eq'

    for word, i in tokenizer.word_index.items():
      if i == pred:
        pred_word = word

    start = start + ' '+ pred_word
    if pred_word == 'eq':
      break
  return start



def create_caption_beam(model, tokenizer, feature, max_len, k):
  start = 'sq'
  start = tokenizer.texts_to_sequences([start])[0]
  depth_seq=[]
  for word, num in tokenizer.word_index.items():
    if word == 'eq':
      end = num
  #end일때 더이상 따지지 않기 위해 end를 찾아놓음
  sequences = [[start, 0.0]]
  max_len_flag=list()
  end_flag =0

  depth_seq=[]
  while len(sequences[0][0]) < max_len :
    all_candidate = list()
    depth =1
    for sequence in sequences:
      seq = pad_sequences([sequence[0]],maxlen = max_len)
      

      all_pred = model.predict([feature, seq],verbose = 0)

      candidate_preds = np.argsort(all_pred[0])[-k:]

      for candidate in candidate_preds: 
        get_cap = sequence[0][:]
        score = sequence[1]
        get_cap.append(candidate)
        score = score+np.log(all_pred[0][candidate]+1e-10)
        if get_cap[-1] == end :
          #depth만큼 나눠 평균을 구해준다.
          depth_seq.append([get_cap,score/depth])
          continue
        else :  
          all_candidate.append([get_cap,score])
    sequences = all_candidate
    sequences = sorted(sequences, key = lambda I: I[1])
    sequences = sequences[-k:]

  depth_seq = sorted(depth_seq, key = lambda l: l[1])
  depth_seq = depth_seq[-1][0]
  #최고점수 시퀀스 찾음
  text = ''
  for t in depth_seq:
    for word,i in tokenizer.word_index.items():
      if i==t:
        if word != 'eq' or word != 'sq':
          text = text+' '+word
        elif word == 'eq':
          break

  return text



def Bleu_Score(model, seq,sortedList, feature, tokenizer,saved_path, max_len):
  y = list()
  pred = list()

  for id in sortedList:
    f_arr = np.array(feature[id][:])
    generated = create_caption_greedy(model, tokenizer, f_arr, max_len)
    target_words = [line.split() for line in seq[id]]
    y.append(target_words)
    pred.append(generated.split())

  print('greedy')
  print('BLEU-1: ' + str(corpus_bleu(y,pred,weights=(1.0,0,0,0))))
  print('BLEU-2: ' + str(corpus_bleu(y,pred,weights=(0.5,0.5,0,0))))
  print('BLEU-3: ' + str(corpus_bleu(y,pred,weights=(0.3,0.3,0.3,0))))
  print('BLEU-4: ' + str(corpus_bleu(y,pred,weights=(0.25, 0.25, 0.25, 0.25))))
  with open(saved_path, 'w') as f:
    f.write('greedy\n')
    f.write('BLEU-1: ' + str(corpus_bleu(y,pred,weights=(1.0,0,0,0)))+'\n')
    f.write('BLEU-2: ' + str(corpus_bleu(y,pred,weights=(0.5,0.5,0,0)))+'\n')
    f.write('BLEU-3: ' + str(corpus_bleu(y,pred,weights=(0.3,0.3,0.3,0)))+'\n')
    f.write('BLEU-4: ' + str(corpus_bleu(y,pred,weights=(0.25, 0.25, 0.25, 0.25)))+'\n')
  
def k_beam_Bleu_Score(model, seq,sortedList, feature, tokenizer,saved_path, max_len, k):
  y = list()
  pred = list()

  for id in sortedList:
    f_arr = np.array(feature[id][:])
    generated = create_caption_beam(model, tokenizer, f_arr, max_len, k)
    target_words = [line.split() for line in seq[id]]
    y.append(target_words)
    pred.append(generated.split())

  print('beam')
  print('BLEU-1: ' + str(corpus_bleu(y,pred,weights=(1.0,0,0,0))))
  print('BLEU-2: ' + str(corpus_bleu(y,pred,weights=(0.5,0.5,0,0))))
  print('BLEU-3: ' + str(corpus_bleu(y,pred,weights=(0.3,0.3,0.3,0))))
  print('BLEU-4: ' + str(corpus_bleu(y,pred,weights=(0.25, 0.25, 0.25, 0.25))))
  with open(saved_path, 'w') as f:
    f.write(str(k)+'beam\n')
    f.write('BLEU-1: ' + str(corpus_bleu(y,pred,weights=(1.0,0,0,0)))+'\n')
    f.write('BLEU-2: ' + str(corpus_bleu(y,pred,weights=(0.5,0.5,0,0)))+'\n')
    f.write('BLEU-3: ' + str(corpus_bleu(y,pred,weights=(0.3,0.3,0.3,0)))+'\n')
    f.write('BLEU-4: ' + str(corpus_bleu(y,pred,weights=(0.25, 0.25, 0.25, 0.25)))+'\n')
  
    
  


if __name__ == '__main__':
  
  
  base_directory = '/content/gdrive/My Drive/Colab Notebooks/showandtell'
  saved_data_path = os.path.join(base_directory,'data')
  data_h5_paths = os.path.join(saved_data_path, 'needs.hdf5')
  needs = h5py.File(data_h5_paths, 'r')

  test_dataset_list_path = needs['test_code_path'][()]
  
  test_feature_path = needs['test_feature_path'][()]

  test_seq_path_X = needs['test_seq_path_X'][()]
  test_seq_path_Y = needs['test_seq_path_Y'][()]
  
  tokenpath = needs['token_path'][()]
  max_len = needs['max_len'][()]
  vocab_size= needs['vocab_size'][()]
  
  req_token_path = needs['req_token_path'][()]
  req_test_list_path = needs['req_test_list_path'][()]
  req_seq_path = needs['req_seq_path'][()]
  
  model_path = os.path.join(base_directory,'real_saved_model','model_39_vacc_0.372_vloss_3.722_acc0.538_loss2.069.h5')
  saved_path = os.path.join(base_directory,'data','greedy.txt')
  beam_saved_path = os.path.join(base_directory,'data','k=3beam.txt')
  
  
  model = load_model(model_path)

  with open(req_token_path, 'rb') as handle:
    tokenizer = pickle.load(handle)
  with open(req_test_list_path, 'rb') as handle:
    test_list = pickle.load(handle)
  with open(req_seq_path, 'rb') as handle:
    sequence = pickle.load(handle)
  with h5py.File(test_feature_path,'r') as hf:
    model = load_model(model_path)
    Bleu_Score(model, sequence,test_list, hf, tokenizer, saved_path, max_len)
    k_beam_Bleu_Score(model, sequence,test_list, hf, tokenizer,beam_saved_path, max_len, 3)



greedy
BLEU-1: 0.5714937980366791
BLEU-2: 0.3440705692607763
BLEU-3: 0.244745379753475
BLEU-4: 0.12625593506226632
beam
BLEU-1: 0.6128750481633843
BLEU-2: 0.3630172269816211
BLEU-3: 0.2559594519059041
BLEU-4: 0.13715272634208575
